# Doing Point cloud data extraction and token attribution (Circum tutorial) for the point cloud dataset of noto region of Japan.

This are tutorials for analysing the pointcloud dataset released by the geospatial organisation of Japan for the region of Noto Peninsula before the earthquake.

This will demonstrate our E2E pipeline for the users providing how they can
- Parse the various tiles from the dataset and get the initial metadata of the pointcloud.
- Perform the preprocessing steps to get the pointcloud in a format that can be used for further steps processing.
- After cropping and generating the new lidar file for doing the tiling process (optional).
- And finally attributing the tokens in for data providers for their contribution to the protocol.



In [ ]:
# installing all the requisite dependencies of the data_preparation project.
## %pip install  -r ../requirements.txt

In [5]:


import laspy
import numpy as np
import pandas as pd
import os

params_list = []

def read_data_header(base_dir = '../datas/'):
    for it in os.listdir(base_dir):
        try:
            file_path = os.path.join(base_dir, it)
            read_file = laspy.read(file_path)
            dimensions = (str(read_file.header.x_max), str(read_file.header.x_min) ,str(read_file.header.y_max) ,str(read_file.header.y_min), str(read_file.header.z_max) ,str(read_file.header.z_min) )
            point_amount = (str(read_file.header.point_count))
            maxs_mins = (str(read_file.header.maxs), str(read_file.header.mins))
            scale = (str(read_file.header.scale[0]))
            point_dimensions = (read_file.point_format.dimensions)  
            #user_data = read_file['user_data']          
            
            params_list.append({
            "filename": it,
            "dimensions": dimensions,
            "point_amount": point_amount,
            "maxs_mins": maxs_mins,
            "scale": scale,
            "point_dimensions": point_dimensions,
            })    
            
            # print('dimensions' + ' ' + str(read_file.header.x_max) + ' ' + str(read_file.header.x_min) + ' ' + str(read_file.header.y_max) + ' ' + str(read_file.header.y_min))
            # print('number of points' + ' ' + str(read_file.header.point_count))
            # print('header maxs and mins ' + '...' + str(read_file.header.maxs) + '...' + str(read_file.header.mins))
            # print('scale:' + str(read_file.header.scale[0]))
            # print('offset:' + str(read_file.header.offset[0]))
            # print('crs code:' + str(read_file))
            # print ('dimensions:' + str(read_file.point_format.dimensions))
        except Exception as e:
            print( f"error in getting values for file {it}: " +str(e))
    df = pd.DataFrame(params_list)
    return df
    
print(read_data_header())

error in getting values for file 07ED5834.las: buffer size must be a multiple of element size
error in getting values for file japan_ne.zip: Invalid file signature "b'PK\x03\x04'"
        filename                                         dimensions  \
0   07ED4942.las  (-0.01, -1000.0, 166499.99, 165750.0, 242.1400...   
1   07ED5814.las  (-6000.01, -7000.0, 164249.99, 163500.0, 197.6...   
2   07ED4933.las  (-3000.01, -4000.0, 165749.99, 165000.0, 183.0...   
3   07ED5821.las  (-5000.01, -6000.0, 164999.99, 164250.0, 123.2...   
4   07ED4844.las  (-4000.01, -5000.0, 165749.99, 165000.0, 91.42...   
5   07ED5824.las  (-4000.01, -5000.0, 164249.99, 163500.0, 329.9...   
6   07ED5822.las  (-4000.01, -5000.0, 164999.99, 164250.0, 254.1...   
7   07ED4934.las  (-2000.01, -3000.0, 165749.99, 165000.0, 274.5...   
8   07ED5832.las  (-6000.01, -7000.0, 163499.99, 162750.0, 262.3...   
9   07ED5831.las  (-7000.01, -8000.0, 163499.99, 162750.0, 49.96...   
10  07ED5823.las  (-5000.01, -6000.0, 1

In [3]:
## now importing for one of the given las file.
 
def parameters_header(lidar_file_path = "../datas/07ED4844.las"):
    """
    This function reads the las file header and returns the parameters as a dictionary
    """
    params = []
    with laspy.open(lidar_file_path) as fh:
        print(f"file details: major-version: {fh.header.major_version}")
        params = {
            
                "x_min" : fh.header.x_min,
                "x_max" : fh.header.x_max,
                "x_scale" : fh.header.x_scale,
                "x_offset" : fh.header.x_offset,
                "y_min" : fh.header.y_min,
                "y_max" : fh.header.y_max,
                "y_scale" : fh.header.y_scale,
                "y_offset" : fh.header.y_offset,
                "z_min" : fh.header.z_min,
                "z_max" : fh.header.z_max,
                "z_scale" : fh.header.z_scale,
                "z_offset" : fh.header.z_offset,
            }
        
        return pd.DataFrame(params, index=[0])

df_init = parameters_header()

print("-----------------------------------------------------")
print(f"scaling values:  {df_init['x_scale'][0], df_init['y_scale'][0], df_init['z_scale'][0]}")
def scaling_parameters(x,y,z, df_init):
    """
    fetches all the sides of the X,Y,Z axis and returns them as a dictionary
    """    
     
    #if (x + df_init['x_scale'][0]) > df_init['']     
    X_coord = (x * df_init['x_scale'][0]) + df_init['x_offset'][0],
    Z_coord = (z * df_init["z_scale"][0]) + df_init["z_offset"][0],
    Y_coord= (y * df_init["y_scale"][0]) + df_init["y_offset"][0],
    return X_coord,Y_coord,Z_coord

print("example of scaling of the coordinates: " + str(scaling_parameters(35.447227, 136.756165,100, df_init)))

file details: major-version: 1
-----------------------------------------------------
scaling values:  (0.01, 0.01, 0.01)
example of scaling of the coordinates: ((0.35447227,), (1.36756165,), (1.0,))


In [6]:

def determine_correct_indices(lidar_file_path):
    """
    defines the indices that are not valid for the point cloud.
    """
    with laspy.open(lidar_file_path) as read_file:
        ## define the current in-valid values for the parameters:
        X_invalid = (read_file.header.mins[0] > abs(df_init['x_max'][0])) | (read_file.header.maxs[0] < df_init['x_min'][0])
        Y_invalid = (read_file.header.mins[1] > abs(df_init['y_max'][0])) | (read_file.header.maxs[1] < df_init['y_min'][0])
        Z_invalid = (read_file.header.mins[2] > abs(df_init['z_max'][0])) | (read_file.header.maxs[2] < df_init['z_min'][0])
        bad_indices = np.where(X_invalid | Y_invalid | Z_invalid)

    print(bad_indices)


## Now running the cropping step:

In [ ]:
import open3d as o3d
import geopandas as gpd
from shapely.geometry import Point  
import shapely as sh
from pyproj import Transformer
from geopandas import GeoDataFrame, points_from_xy
Points1 = Point(float(37.124), float(136.54), float(1.0))
Points2 = Point(float(38.124), float(137.54), float(1.0))
transformer = Transformer.from_crs('EPSG:4326','EPSG:6685') ## for GPS coordiantes to japan coordinates

## taking exampl of the points that are considered as boundation points 
X1,Y1,Z1 = transformer.transform(Points1.x, Points1.y, Points1.z)
X2,Y2,Z2 = transformer.transform(Points2.x, Points2.y, Points2.z)

## scaling to thelocal coordinates
X1_scaled, Y1_scaled, Z1_scaled = scaling_parameters(X1,Y1,Z1,df_init)
X2_scaled, Y2_scaled, Z2_scaled = scaling_parameters(X2,Y2,Z2,df_init)


buffer_radius = 0.3

print(f"finally scaled params: X_Scaled: {X1_scaled} to {X2_scaled}, Y_scaled:  {Y1_scaled} to {Y2_scaled} , Z_scaled: {Z1_scaled} to {Z2_scaled} ")

#polygon = Polygon([(X1_scaled, Y1_scaled), (X1_scaled, Y2_scaled), (X2_scaled, Y1_scaled), (X2_scaled, Y2_scaled)])

print("so now applying the cropping technique  using geopandas")

lidar_file_path = "../datas/07ED4844.las"

fileInfo = laspy.read(lidar_file_path)

x_cropped_region = (fileInfo.header.mins[0] <= X1) & (fileInfo.header.maxs[0] >= X1)
y_cropped_region = (fileInfo.header.mins[1] <= Y1) & (fileInfo.header.maxs[1] >= Y1)
z_cropped_region = (fileInfo.header.mins[2] <= Z1) & (fileInfo.header.maxs[2] >= Z1)


cropped_region = np.where(x_cropped_region & y_cropped_region & z_cropped_region)

cropped_points = fileInfo.points[cropped_region].copy()

resulting_file = laspy.LasData(fileInfo.header)
resulting_file.points = cropped_points

print("showcasing the output points matrice" + str(resulting_file.points))
resulting_file.write("good_points.las")


# lidar_df = gpd.GeoDataFrame({
#     "x" :  fileInfo.x,
#     "y" :  fileInfo.y,
#     "z" :  fileInfo.z,
#     "geometry": gpd.points_from_xy(fileInfo.x, fileInfo.y, fileInfo.z)
# })

# ## defining the point of intersection points
# df_poi = gpd.GeoDataFrame({"geometry": [Points1]}, crs="EPSG:4326")

# ## doing the transformation of the point cloud dataset 

# joined_dataframe = gpd.sjoin(lidar_df, df_poi, how="left", op="within")


# intersecting_points = joined_dataframe.sample_points
# print(f"intersection point with buffer :{intersecting_points}" )

# output_las = laspy.LasData(header=fileInfo.header)
# print(f"number of points: {output_las.xyz.shape}")



# output_las.write("intersecting_points_2.las")


## Now comes the reconstruction step(optional) : 

In the gepspatial pipeline, we rarely get a well defined pointcloud from the start.

